In [1]:
import requests
import json
#url="https://maps.googleapis.com/maps/api/staticmap?center=Brooklyn+Bridge,New+York,NY&zoom=13&size=600x300&maptype=roadmap&markers=color:blue%7Clabel:S%7C40.702147,-74.015794&markers=color:green%7Clabel:G%7C40.711614,-74.012318&markers=color:red%7Clabel:C%7C40.718217,-73.998284&"
#resp=requests.get(url)
#resp.text
key="gxy7IYotnmfdd1SrvXfy1hfG5z4HfU0pGbbVPclKWAy07uqNviLGHrnzp0J_Nt1hJ_vQd4AnIX3LQzh1SV9vybXQpR_MpvhhZTIUrx8b6K0Qp4yfU0wrPDQE_wDXXXYx"
#def yelp(food):
user_id="lUrLPc8-KXBjLODVZRvkRQ"
url='https://api.yelp.com/v3/autocomplete?text=del&latitude=37.786882&longitude=-122.399972'
#print(url)
data={'Authorization': 'lUrLPc8-KXBjLODVZRvkRQ'}
response = requests.get(url,params="data")
#return(response.json())
#user_text=input("")
#food = { 'term' : user_text,'latitude': 43.03821555,'longitude': -76.1333456417294,'radius': 40000}
#yelp(food)
response.json()

{'error': {'code': 'VALIDATION_ERROR',
  'description': 'Authorization is a required parameter.',
  'field': 'Authorization',
  'instance': None}}

In [8]:
from urllib.parse import quote
import json
import pandas as pd
import requests
import folium

API_KEY= 'gxy7IYotnmfdd1SrvXfy1hfG5z4HfU0pGbbVPclKWAy07uqNviLGHrnzp0J_Nt1hJ_vQd4AnIX3LQzh1SV9vybXQpR_MpvhhZTIUrx8b6K0Qp4yfU0wrPDQE_wDXXXYx'

API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  
SEARCH_LIMIT = 10

def request(host, path, api_key, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location):

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

get_type=input("Enter the type of the restaurant(Asian,French,Japanese...etc.): ")
get_location=input("Enter a location: ")
restaurants=search(API_KEY,get_type,get_location)


name=[]
address=[]
rating=[]
location=[]
for num in restaurants['businesses']:
    name.append(num['name'])
    address.append(num['location']['address1'])
    rating.append(num['rating'])
    location.append((num['coordinates']['latitude'],num['coordinates']['longitude']))
series_dict = { 'Name':  name,'Address': address,'Rating on Yelp': rating}  
show_table=pd.DataFrame(series_dict)
display(show_table)
restaurants


Enter the type of the restaurant(Asian,French,Japanese...etc.): Japanese
Enter a location: new york
Querying https://api.yelp.com/v3/businesses/search ...


,Name,Address,Rating on Yelp
0,Kaikagetsu NYC,162 Orchard St,4.5
1,Benemon,108 E 4th St,4.5
2,Aoi Kitchen,320 E 6th St,4.5
3,Ootoya Chelsea,8 W 18th St,4.0
4,Izakaya Fuku,71-28 Roosevelt Ave,4.5
5,Nami Nori,33 Carmine St,4.0
6,DOMODOMO New York,140 W Houston St,4.5
7,Okonomi,150 Ainslie St,4.5
8,Raku,342 E 6th St,4.5
9,nonono,118 Madison Ave,4.5


{'businesses': [{'id': 'ax4INkzwL__-d_kGJfcA-w',
   'alias': 'kaikagetsu-nyc-new-york-2',
   'name': 'Kaikagetsu NYC',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/m_ARAOwDgHTjn3WkmwkolA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/kaikagetsu-nyc-new-york-2?adjust_creative=lUrLPc8-KXBjLODVZRvkRQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=lUrLPc8-KXBjLODVZRvkRQ',
   'review_count': 52,
   'categories': [{'alias': 'japanese', 'title': 'Japanese'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.72104, 'longitude': -73.98848},
   'transactions': ['restaurant_reservation'],
   'location': {'address1': '162 Orchard St',
    'address2': '',
    'address3': None,
    'city': 'New York',
    'zip_code': '10002',
    'country': 'US',
    'state': 'NY',
    'display_address': ['162 Orchard St', 'New York, NY 10002']},
   'phone': '+16465903900',
   'display_phone': '(646) 590-3900',
   'distance': 1800.6650070425846},
  {'id': 'ax

In [70]:
#businesses:{name:xx,location:{address1:xx,address2:xx},rating:xx,city:xx,}


In [9]:
def get_coordinates(search):
    url = 'https://nominatim.openstreetmap.org/search' 
    options = { 'q' : search, 'format' : 'json'}
    response = requests.get(url, params = options)            
    geodata = response.json()
    coords = { 'lat' : float(geodata[0]['lat']), 'lng' : float(geodata[0]['lon']) }
    return coords['lat'],coords['lng']

coords = get_coordinates(get_location)
show_map = folium.Map(location=coords, zoom_start=13)
d={'a':name,'b':rating,'c':address,'d':location}
for a in range(SEARCH_LIMIT):
    loc=(d['d'][a])
    message=f"Restaurant name:{d['a'][a]} Rating:{d['b'][a]} Address:{d['c'][a]}"
    marker = folium.Marker(location=loc,popup=message)
    show_map.add_child(marker)
    
    
show_map
    